In [1]:
import time
from pprint import pprint
from tqdm import tqdm
import openai

#avenieca sdk imports
from avenieca import Signal
from avenieca.producers import Event
from avenieca.api.model import *
from avenieca.api.eca import ECA

#local file imports
from config import *
import utils as util

load_dotenv()
data_path = os.getenv("DATA_PATH")
url = '%s/gwp_data.csv' % data_path

def prettyprint(res, status=None):
    try:
        pprint(res.__dict__)
    except:
        print(res)
    if status:
        print(status)

In [2]:
import pandas as pd
data = pd.read_csv(url)

### Write directly to the ECA database

In [3]:
date = data['date'].values
quarter = data['quarter'].values
department = data['department'].values
day = data['day'].values
team = data['team'].values
smv = data['smv'].values
wip = data['wip'].values
incentive = data['targeted_productivity'].values
now = data['no_of_workers'].values
actual_prod = data['actual_productivity'].values
eca_server = os.getenv("ECA_SERVER")
eca_secret = os.getenv("ECA_SECRET")
openai.api_key = openai_key

config = Config(uri=eca_server, username=username, password=password)
eca = ECA(config=config)

for i in tqdm(range(0, len(date))):
    date_ess = util.create_ess_and_sequence(eca, date[i], eca_secret, date_twin_config, openai, True)
    quarter_ess = util.create_ess_and_sequence(eca, quarter[i], eca_secret, quarter_twin_config, openai, True)
    department_ess = util.create_ess_and_sequence(eca, department[i], eca_secret, department_twin_config, openai, True)
    day_ess = util.create_ess_and_sequence(eca, day[i], eca_secret, day_twin_config, openai, True)
    team_ess = util.create_ess_and_sequence(eca, [float(team[i])], eca_secret, team_twin_config)
    smv_ess = util.create_ess_and_sequence(eca, [float(smv[i])], eca_secret, smv_twin_config)
    wip_ess = util.create_ess_and_sequence(eca, [float(wip[i])], eca_secret, wip_twin_config)
    incentive_ess = util.create_ess_and_sequence(eca, [float(incentive[i])], eca_secret, incentive_twin_config)
    now_ess = util.create_ess_and_sequence(eca, [float(now[i])], eca_secret, now_twin_config)
    gwp_aggregate_in_twins = [
        date_ess,
        quarter_ess,
        department_ess,
        day_ess,
        team_ess,
        smv_ess,
        wip_ess,
        now_ess,
        incentive_ess
    ]
    gwp_aggregate_insert = ESSInsert(
        module_id=aggregate_config.module_id,
        state=[],
        valence=0.0,
    )
    gwp_aggregate_insert = util.create_aggregate_from_ess(gwp_aggregate_in_twins, gwp_aggregate_insert)
    gwp_aggregate = util.search_insert_ess_and_sequence(eca, gwp_aggregate_insert)
    actual_prod_ess = util.create_ess_and_sequence(eca, [float(actual_prod[i])], eca_secret, actual_prod_twin_config)

    gwp_record_in_twins = [
        gwp_aggregate,
        actual_prod_ess
    ]
    gwp_record_insert = ESSInsert(
        module_id=gwp_record_config.module_id,
        state=[],
        valence=0.0
    )
    gwp_record_insert = util.create_aggregate_from_ess(gwp_record_in_twins, gwp_record_insert)
    util.search_insert_ess_and_sequence(eca, gwp_record_insert)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 691/691 [03:16<00:00,  3.52it/s]


### Retrieving from Multi-level aggregates

In [4]:
res, status = eca.ess.search(data=Search(
    module_id="team",
    state=[8.0],
    limit=3
))
prettyprint(res[0])

{'ess': {'aggregate_context': [],
         'aggregate_emb_inp': [],
         'aggregate_id': [],
         'aggregate_module_id': [],
         'aggregate_score': [],
         'aggregate_shape': [],
         'aggregate_valence': [],
         'avg_ess_score': 0,
         'avg_ess_valence': 0.0,
         'context': None,
         'created_at': '2023-05-31T15:51:31.518640Z',
         'embedding_input': None,
         'id': 1,
         'module_id': 'team',
         'score': 56,
         'state': [8.0],
         'total_ess_score': 0,
         'updated_at': '2023-05-31T17:16:13.359352Z',
         'valence': 50.0},
 'score': 0.0}


In [5]:
res, status = eca.ess.get_all_aggregates(module_id="team", aggregate_module_id="gwp_aggregate", ess_id=1)
print(len(res))

56


In [6]:
print(res[0].id)

689


In [7]:
res, status = eca.ess.get_one_pretty(module_id="gwp_aggregate", db_id=689)
prettyprint(res)

{'aggregate_context': [None, None, None, None, None, None, None, None, None],
 'aggregate_emb_inp': [59, 2, 1, 6, None, None, None, None, None],
 'aggregate_id': [59, 2, 1, 6, 1, 62, 548, 17, 3],
 'aggregate_module_id': ['gwp_date',
                         'quarter',
                         'department',
                         'day',
                         'team',
                         'smv',
                         'wip',
                         'no_of_workers',
                         'incentive'],
 'aggregate_score': [10, 187, 683, 116, 56, 6, 1, 108, 143],
 'aggregate_shape': [1536, 1536, 1536, 1536, 1, 1, 1, 1, 1],
 'aggregate_valence': [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0],
 'avg_ess_score': 145,
 'avg_ess_valence': 50.0,
 'context': None,
 'created_at': '2023-05-31 16:28:17.604598 UTC',
 'embedding_input': None,
 'id': 689,
 'module_id': 'gwp_aggregate',
 'score': 1,
 'state': ['3/11/2015',
           'Quarter2',
           'sweing',
           'Wedn

In [8]:
res, status = eca.ess.get_all_aggregates(module_id="gwp_aggregate", aggregate_module_id="gwp_record", ess_id=689)
print(res[0].id)

689


In [10]:
print(res[0])

{
  "id": 689,
  "state": [
    -0.020157874,
    0.005088618,
    0.00092155434,
    -0.030879032,
    -0.0017169581,
    0.023054423,
    -0.027261624,
    -0.024876233,
    -0.01896518,
    0.00826368,
    -0.001166483,
    0.0065794885,
    -0.017654523,
    -0.009154925,
    -0.027785886,
    0.009227011,
    0.018244319,
    -0.026147569,
    0.0099544255,
    -0.0019250246,
    0.010832564,
    0.0026245867,
    -0.0010837478,
    0.0036305145,
    -0.001001832,
    0.006972685,
    0.004590569,
    -0.023618003,
    0.02097048,
    -0.02899169,
    0.026842216,
    -0.017693844,
    -0.011828662,
    -0.005308153,
    -0.023945667,
    -0.0005222141,
    -0.0026557148,
    -0.019319056,
    0.02272676,
    -0.0023575407,
    0.015360877,
    0.0033077656,
    0.013447321,
    0.0031701468,
    -0.03187513,
    0.027759673,
    -0.019712253,
    -0.02291025,
    -0.04160019,
    0.025334962,
    0.024299545,
    0.033710048,
    -0.029201394,
    -0.031744067,
    -0.011134014,
